## Домашнее задание 7. Обучение без учителя. Решение

В этом задании мы рассмотрим, как работают методы снижения размерности и кластеризации. Заодно ещё раз попрактикуемся в задаче классификации.

Мы будем работать с датасетом [Samsung Human Activity Recognition](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Скачайте данные [здесь](https://drive.google.com/file/d/14RukQ0ylM2GCdViUHBBjZ2imCaYcjlux/view?usp=sharing). Данные получены с акселерометров и гироскопов мобильных телефонов Samsung Galaxy S3, также известен тип активности человека с телефоном в кармане.

Сначала мы сделаем вид, что тип активности нам неизвестен, и попытаемся кластеризовать людей исключительно на основе имеющихся признаков. Затем решим задачу определения типа физической активности как задачу классификации.

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use(['seaborn-v0_8-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

RANDOM_STATE = 17

In [ ]:
# при необходимости измените путь
PATH_TO_SAMSUNG_DATA = "../data"

In [ ]:
X_train = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA, "samsung_train.txt"))
y_train = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA,
                                  "samsung_train_labels.txt")).astype(int)

X_test = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA, "samsung_test.txt"))
y_test = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA,
                                  "samsung_test_labels.txt")).astype(int)

In [ ]:
# Проверяем размерности
assert(X_train.shape == (7352, 561) and y_train.shape == (7352,))
assert(X_test.shape == (2947, 561) and y_test.shape == (2947,))

Для кластеризации вектор целевой переменной не нужен, поэтому будем работать с объединением обучающей и тестовой выборок. Объедините `X_train` с `X_test`, и `y_train` с `y_test`.

In [ ]:
X = np.vstack([X_train, X_test])
y = np.hstack([y_train, y_test])

Определите количество уникальных значений меток целевого класса.

In [ ]:
np.unique(y)

In [ ]:
n_classes = np.unique(y).size

[Эти метки соответствуют:](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.names)
- 1 – ходьба
- 2 – подъём по лестнице
- 3 – спуск по лестнице
- 4 – сидение
- 5 – стояние
- 6 – лежание

Отмасштабируйте выборку с помощью `StandardScaler` с параметрами по умолчанию.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Снизьте число измерений с помощью PCA, оставив столько компонент, сколько необходимо для объяснения как минимум 90% дисперсии исходных (масштабированных) данных. Используйте масштабированный датасет и зафиксируйте `random_state` (константа RANDOM_STATE).

In [ ]:
pca = PCA(n_components=0.9, random_state=RANDOM_STATE).fit(X_scaled)
X_pca = pca.transform(X_scaled)

**Вопрос 1:**

Какое минимальное число главных компонент необходимо для покрытия 90% дисперсии исходных (масштабированных) данных?

In [ ]:
X_pca.shape

**Ответ:** 65

**Вопрос 2:**

Какой процент дисперсии покрывается первой главной компонентой? Округлите до ближайшего процента.

**Ответ:** 51

In [ ]:
round(float(pca.explained_variance_ratio_[0] * 100))

Визуализируйте данные в проекции на первые две главные компоненты.

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, s=20, cmap='viridis');

**Вопрос 3:**

Если всё сделано правильно, вы увидите несколько кластеров, почти идеально разделённых друг от друга. Какие типы активности входят в эти кластеры?

**Ответ:** 2 кластера: (ходьба, подъём по лестнице, спуск по лестнице) и (сидение, стояние, лежание)

---

Выполните кластеризацию методом `KMeans`, обучив модель на данных со сниженной размерностью (после PCA).

Параметры:
- **n_clusters** = n_classes
- **n_init** = 100
- **random_state** = RANDOM_STATE

In [ ]:
kmeans = KMeans(n_clusters=n_classes, n_init=100, 
                random_state=RANDOM_STATE)
kmeans.fit(X_pca)
cluster_labels = kmeans.labels_

Визуализируйте данные в проекции на первые две главные компоненты, раскрасив точки по кластерам.

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, s=20,  
            cmap='viridis');

Посмотрите на соответствие между метками кластеров и исходными метками классов.

In [ ]:
tab = pd.crosstab(y, cluster_labels, margins=True)
tab.index = ['ходьба', 'подъём по лестнице',
            'спуск по лестнице', 'сидение', 'стояние', 'лежание', 'всего']
tab.columns = ['cluster' + str(i + 1) for i in range(6)] + ['всего']
tab

**Вопрос 4:**

Какая активность лучше всего отделяется от остальных по описанной выше метрике?

**Ответ:** все три варианта неверны (лучше всего отделяется «подъём по лестнице»)

In [ ]:
pd.Series(tab.iloc[:-1,:-1].max(axis=1).values / 
          tab.iloc[:-1,-1].values, index=tab.index[:-1])

Видно, что KMeans не очень хорошо различает активности. Используйте метод «локтя» для выбора оптимального числа кластеров.

In [ ]:
inertia = []
for k in tqdm_notebook(range(1, n_classes + 1)):
    kmeans = KMeans(n_clusters=k, n_init=100, 
                    random_state=RANDOM_STATE).fit(X_pca)
    inertia.append(np.sqrt(kmeans.inertia_))

In [ ]:
plt.plot(range(1, 7), inertia, marker='s');

Вычислим $D(k)$, как описано в [статье](https://www.kaggle.com/kashnitsky/topic-7-unsupervised-learning-pca-and-clustering) в разделе «Выбор числа кластеров для KMeans».

In [ ]:
d = {}
for k in range(2, 6):
    i = k - 1
    d[k] = (inertia[i] - inertia[i + 1]) / (inertia[i - 1] - inertia[i])

In [ ]:
d

**Вопрос 5:**

Сколько кластеров можно выбрать по методу «локтя»?

**Ответ:** 2

---

Попробуем другой алгоритм кластеризации — агломеративную кластеризацию.

In [ ]:
ag = AgglomerativeClustering(n_clusters=n_classes, 
                             linkage='ward').fit(X_pca)

Рассчитайте Adjusted Rand Index (`sklearn.metrics`) для полученной кластеризации и для `KMeans`.

In [ ]:
print('KMeans: ARI =', metrics.adjusted_rand_score(y, cluster_labels))
print('Agglomerative Clustering: ARI =', 
      metrics.adjusted_rand_score(y, ag.labels_))

**Вопрос 6:**

Выберите все верные утверждения.

**Ответ:**
- По ARI, KMeans справился с кластеризацией хуже, чем агломеративная кластеризация — **верно**
- Для ARI неважно, какие метки присвоены кластеру, важно лишь разбиение объектов на кластеры — **верно**
- В случае случайного разбиения на кластеры ARI будет близок к нулю — **верно**

---

Теперь решим задачу классификации, учитывая, что данные размечены.

Для классификации используем метод опорных векторов — класс `sklearn.svm.LinearSVC`.

Выберите гиперпараметр `C` для `LinearSVC` с помощью `GridSearchCV`.

- Обучите новый `StandardScaler` на обучающей выборке, примените масштабирование к тестовой
- В `GridSearchCV` укажите `cv` = 3.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
svc = LinearSVC(random_state=RANDOM_STATE)
svc_params = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [ ]:
%%time
best_svc = GridSearchCV(svc, svc_params, n_jobs=1, cv=3, verbose=1)
best_svc.fit(X_train_scaled, y_train);

In [ ]:
best_svc.best_params_, best_svc.best_score_

**Вопрос 7:**

Какое значение гиперпараметра `C` оказалось лучшим?

**Ответ:** 0.1

In [ ]:
y_predicted = best_svc.predict(X_test_scaled)

In [ ]:
tab = pd.crosstab(y_test, y_predicted, margins=True)
tab.index = ['ходьба', 'подъём по лестнице',
             'спуск по лестнице', 'сидение', 'стояние', 'лежание', 'всего']
tab.columns = ['ходьба', 'подъём по лестнице',
             'спуск по лестнице', 'сидение', 'стояние', 'лежание', 'всего']
tab

Как видим, задача классификации решается достаточно хорошо.

**Вопрос 8:**

Путает ли SVM классы внутри групп активностей, выявленных ранее (в вопросе 3)?

**Ответ:** Да. Классификатор решил задачу хорошо, но не идеально.

Наконец, сделайте то же самое, что и в вопросе 7, но с применением PCA.

**Вопрос 9:**

Какова разница между лучшим качеством (accuracy) по кросс-валидации при использовании всех 561 исходных признаков и при применении PCA? Округлите до ближайшего процента.

**Ответ:** 4%

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=0.9, random_state=RANDOM_STATE)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
svc = LinearSVC(random_state=RANDOM_STATE)
svc_params = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [ ]:
%%time
best_svc_pca = GridSearchCV(svc, svc_params, n_jobs=1, cv=3, verbose=1)
best_svc_pca.fit(X_train_pca, y_train);

In [ ]:
best_svc_pca.best_params_, best_svc_pca.best_score_

Результат с PCA хуже на 4% по доле правильных ответов на кросс-валидации.

In [ ]:
round(100 * (best_svc_pca.best_score_ - best_svc.best_score_))

**Вопрос 10:**

Выберите все верные утверждения:

**Ответ:**
- PCA можно использовать для визуализации данных, но для этой задачи есть лучшие методы, например t-SNE. Однако PCA имеет меньшую вычислительную сложность — **верно**
- PCA строит линейные комбинации исходных признаков, и в некоторых приложениях они могут быть плохо интерпретируемы человеком — **верно**

### Комментарий:
1. PCA позволила значительно сократить время обучения модели, но качество пострадало не так сильно — всего на 4%
2. Для визуализации многомерных данных лучше использовать методы manifold learning, в частности t-SNE
3. Линейные комбинации признаков, которые строит PCA, плохо интерпретируются человеком, например: 0.574 * salary + 0.234 * num_children
4. SVM и KMeans в принципе не следует сравнивать напрямую — они решают разные задачи